In [105]:
from pyspark.sql import SparkSession
import pyspark

spark = SparkSession.builder.appName('test').getOrCreate()

In [106]:
spark

In [107]:
df = spark.read.option('header', 'false') \
               .option('delimiter', '|') \
               .option('inferSchema', 'true') \
               .csv('/home/new-user/Documents/new/data/nyse_all/nyse_stocks/companylist_noheader.csv')


In [108]:
df.show()

+----+--------------------+------+-----------------+---+----+-----------------+--------------------+--------------------+----+
| _c0|                 _c1|   _c2|              _c3|_c4| _c5|              _c6|                 _c7|                 _c8| _c9|
+----+--------------------+------+-----------------+---+----+-----------------+--------------------+--------------------+----+
| DDD|3D Systems Corpor...| 51.37|  5.34093160546E9|n/a| n/a|       Technology|Computer Software...|http://www.nasdaq...|NULL|
| MMM|          3M Company|142.39|9.316270806833E10|n/a| n/a|      Health Care|Medical/Dental In...|http://www.nasdaq...|NULL|
|WBAI|     500.com Limited| 33.71|  1.10817057295E9|n/a|2013|Consumer Services|Services-Misc. Am...|http://www.nasdaq...|NULL|
|WUBA|         58.com Inc.| 41.57|  3.30225204322E9|n/a|2013|       Technology|Computer Software...|http://www.nasdaq...|NULL|
| AHC|A.H. Belo Corpora...| 11.67|   2.5692000975E8|n/a| n/a|Consumer Services|Newspapers/Magazines|http://www.

In [109]:
new_columns = [
    "Symbol", "CompanyName", "Price", "MarketCap", 
    "Column5", "Column6", "Sector", "Industry", "Website", "Extra"
]

df = df.toDF(*new_columns)


In [110]:
df = df.drop('Column5')

In [111]:
df = df.drop('Column6')

In [112]:
df = df.drop('Extra')

In [113]:
df.show()

+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|Symbol|         CompanyName| Price|        MarketCap|           Sector|            Industry|             Website|
+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|   DDD|3D Systems Corpor...| 51.37|  5.34093160546E9|       Technology|Computer Software...|http://www.nasdaq...|
|   MMM|          3M Company|142.39|9.316270806833E10|      Health Care|Medical/Dental In...|http://www.nasdaq...|
|  WBAI|     500.com Limited| 33.71|  1.10817057295E9|Consumer Services|Services-Misc. Am...|http://www.nasdaq...|
|  WUBA|         58.com Inc.| 41.57|  3.30225204322E9|       Technology|Computer Software...|http://www.nasdaq...|
|   AHC|A.H. Belo Corpora...| 11.67|   2.5692000975E8|Consumer Services|Newspapers/Magazines|http://www.nasdaq...|
|  ATEN|  A10 Networks, Inc.| 11.43|   6.8642274069E8|       Technology|Computer

In [114]:
df.printSchema()

root
 |-- Symbol: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- MarketCap: double (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Website: string (nullable = true)



In [115]:
from pyspark.sql.functions import col

df = df.withColumn("Price", col("Price").cast("double")) 


In [116]:
df.printSchema()

root
 |-- Symbol: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- MarketCap: double (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Website: string (nullable = true)



In [117]:
from pyspark.sql.functions import col, when, count

# List of strings to treat as nulls
null_equivalents = ["n/a", "N/A", "NA", "na", "null", "NULL", ""]

null_counts = df.select([
    count(
        when(
            col(c).isNull() | col(c).isin(null_equivalents),  # check for null or placeholder strings
            c
        )
    ).alias(c) for c in df.columns
])

null_counts.show()



+------+-----------+-----+---------+------+--------+-------+
|Symbol|CompanyName|Price|MarketCap|Sector|Industry|Website|
+------+-----------+-----+---------+------+--------+-------+
|     0|          0|   18|        0|  1019|    1019|      0|
+------+-----------+-----+---------+------+--------+-------+



In [118]:
df.describe().show()

+-------+------+--------------------+-----------------+--------------------+----------------+--------------------+--------------------+
|summary|Symbol|         CompanyName|            Price|           MarketCap|          Sector|            Industry|             Website|
+-------+------+--------------------+-----------------+--------------------+----------------+--------------------+--------------------+
|  count|  3298|                3298|             3280|                3298|            3298|                3298|                3298|
|   mean|   NaN|                NULL|96.36651750000009| 7.925952329822477E9|            NULL|                NULL|                NULL|
| stddev|   NaN|                NULL|3357.623228128144|2.463324980810021E10|            NULL|                NULL|                NULL|
|    min|     A|3D Systems Corpor...|            0.037|                 0.0|Basic Industries|Accident &Health ...|http://www.nasdaq...|
|    max|  ZZJJ|         voxeljet AG|         19

In [119]:
df_new= df.withColumn('Price_double' , df['Price']*2).show()

+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+------------+
|Symbol|         CompanyName| Price|        MarketCap|           Sector|            Industry|             Website|Price_double|
+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+------------+
|   DDD|3D Systems Corpor...| 51.37|  5.34093160546E9|       Technology|Computer Software...|http://www.nasdaq...|      102.74|
|   MMM|          3M Company|142.39|9.316270806833E10|      Health Care|Medical/Dental In...|http://www.nasdaq...|      284.78|
|  WBAI|     500.com Limited| 33.71|  1.10817057295E9|Consumer Services|Services-Misc. Am...|http://www.nasdaq...|       67.42|
|  WUBA|         58.com Inc.| 41.57|  3.30225204322E9|       Technology|Computer Software...|http://www.nasdaq...|       83.14|
|   AHC|A.H. Belo Corpora...| 11.67|   2.5692000975E8|Consumer Services|Newspapers/Magazines|http://www.

In [121]:
df.show()

+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|Symbol|         CompanyName| Price|        MarketCap|           Sector|            Industry|             Website|
+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|   DDD|3D Systems Corpor...| 51.37|  5.34093160546E9|       Technology|Computer Software...|http://www.nasdaq...|
|   MMM|          3M Company|142.39|9.316270806833E10|      Health Care|Medical/Dental In...|http://www.nasdaq...|
|  WBAI|     500.com Limited| 33.71|  1.10817057295E9|Consumer Services|Services-Misc. Am...|http://www.nasdaq...|
|  WUBA|         58.com Inc.| 41.57|  3.30225204322E9|       Technology|Computer Software...|http://www.nasdaq...|
|   AHC|A.H. Belo Corpora...| 11.67|   2.5692000975E8|Consumer Services|Newspapers/Magazines|http://www.nasdaq...|
|  ATEN|  A10 Networks, Inc.| 11.43|   6.8642274069E8|       Technology|Computer

In [122]:
df.na.drop().show()

+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|Symbol|         CompanyName| Price|        MarketCap|           Sector|            Industry|             Website|
+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|   DDD|3D Systems Corpor...| 51.37|  5.34093160546E9|       Technology|Computer Software...|http://www.nasdaq...|
|   MMM|          3M Company|142.39|9.316270806833E10|      Health Care|Medical/Dental In...|http://www.nasdaq...|
|  WBAI|     500.com Limited| 33.71|  1.10817057295E9|Consumer Services|Services-Misc. Am...|http://www.nasdaq...|
|  WUBA|         58.com Inc.| 41.57|  3.30225204322E9|       Technology|Computer Software...|http://www.nasdaq...|
|   AHC|A.H. Belo Corpora...| 11.67|   2.5692000975E8|Consumer Services|Newspapers/Magazines|http://www.nasdaq...|
|  ATEN|  A10 Networks, Inc.| 11.43|   6.8642274069E8|       Technology|Computer

In [128]:
df.filter('Price>80').count()

281

In [ ]:
df.groupBy('Sector').count().show()

+--------------------+-----+
|              Sector|count|
+--------------------+-----+
|       Miscellaneous|   51|
|         Health Care|  111|
|    Public Utilities|  215|
|              Energy|  246|
|Consumer Non-Dura...|  119|
|             Finance|  391|
|                 n/a| 1019|
|    Basic Industries|  208|
|       Capital Goods|  193|
|          Technology|  179|
|   Consumer Services|  443|
|   Consumer Durables|   66|
|      Transportation|   57|
+--------------------+-----+



In [134]:
df.groupby('Sector').agg({"Price" : 'max'}).show()

+--------------------+----------+
|              Sector|max(Price)|
+--------------------+----------+
|       Miscellaneous|    256.83|
|         Health Care|    213.45|
|    Public Utilities|    114.97|
|              Energy|    210.15|
|Consumer Non-Dura...|    218.95|
|             Finance|  192300.0|
|                 n/a|   1216.42|
|    Basic Industries|     391.5|
|       Capital Goods|   1129.02|
|          Technology|    183.76|
|   Consumer Services|    679.62|
|   Consumer Durables|    143.22|
|      Transportation|     197.8|
+--------------------+----------+



In [146]:
df_filtered = df.filter(df.Sector != 'n/a')
df_filtered.show()



+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|Symbol|         CompanyName| Price|        MarketCap|           Sector|            Industry|             Website|
+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|   DDD|3D Systems Corpor...| 51.37|  5.34093160546E9|       Technology|Computer Software...|http://www.nasdaq...|
|   MMM|          3M Company|142.39|9.316270806833E10|      Health Care|Medical/Dental In...|http://www.nasdaq...|
|  WBAI|     500.com Limited| 33.71|  1.10817057295E9|Consumer Services|Services-Misc. Am...|http://www.nasdaq...|
|  WUBA|         58.com Inc.| 41.57|  3.30225204322E9|       Technology|Computer Software...|http://www.nasdaq...|
|   AHC|A.H. Belo Corpora...| 11.67|   2.5692000975E8|Consumer Services|Newspapers/Magazines|http://www.nasdaq...|
|  ATEN|  A10 Networks, Inc.| 11.43|   6.8642274069E8|       Technology|Computer

In [147]:
from pyspark.sql.functions import col, when

# Define strings to treat as nulls
null_equivalents = ["n/a", "N/A", "NA", "na", "null", "NULL", ""]

# Step 1: Replace these placeholder strings with actual nulls in all columns
for c in df.columns:
    df = df.withColumn(
        c,
        when(col(c).isin(null_equivalents), None).otherwise(col(c))
    )

# Step 2: Drop rows with nulls in any column
df_clean = df.dropna(how='any')

df_clean.show()



+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|Symbol|         CompanyName| Price|        MarketCap|           Sector|            Industry|             Website|
+------+--------------------+------+-----------------+-----------------+--------------------+--------------------+
|   DDD|3D Systems Corpor...| 51.37|  5.34093160546E9|       Technology|Computer Software...|http://www.nasdaq...|
|   MMM|          3M Company|142.39|9.316270806833E10|      Health Care|Medical/Dental In...|http://www.nasdaq...|
|  WBAI|     500.com Limited| 33.71|  1.10817057295E9|Consumer Services|Services-Misc. Am...|http://www.nasdaq...|
|  WUBA|         58.com Inc.| 41.57|  3.30225204322E9|       Technology|Computer Software...|http://www.nasdaq...|
|   AHC|A.H. Belo Corpora...| 11.67|   2.5692000975E8|Consumer Services|Newspapers/Magazines|http://www.nasdaq...|
|  ATEN|  A10 Networks, Inc.| 11.43|   6.8642274069E8|       Technology|Computer